In [1]:
NUM_FILES = 350
FOLDER_NAME = '../random_samples_formatting'
GPT_SAVED_FILE_NAME = 'formatted_code_gpt'
GPT_SAVED_FOLDER_NAME = 'reformatted_gpt'

In [2]:
# read in files from folder random_cells
random_cells = []

for i in range(NUM_FILES):
    file_name = f'{FOLDER_NAME}/{i}.py'
    with open(file_name, 'r') as f:
        random_cells.append(f.read())

In [4]:
task = r"Format the code delimited by triple backticks according to PEP 8 conventions. Do not add, remove, or change anything else. Structure your response under the following headings: 'Identified formatting issues' (a short paragraph describing the formatting issues) and 'Formatted code' (e.g., ```python\n#full code with all formatting issues fixed\n```)."

In [5]:
# Estimate cost
import sys
sys.path.append("../../")
import utils

def estimate_tokens():
    in_tok = ''
    out_tok = ''
    for i, cell_src in enumerate(random_cells):
        # estimate prompt
        in_tok += task
        in_tok += f'```python\n{cell_src}\n```"'
        # estimate response
        out_tok += cell_src
    return in_tok, out_tok

in_tok, out_tok = estimate_tokens()

utils.gpt_35_turbo_token_dollar_cost(in_tok, out_tok)

0.23302650000000003

In [6]:
# identify unused functions using vulture (GPT)
import openai
openai.api_key = my_key

# GPT
def format_code(cell_src):
    while True:
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                temperature=0,
                messages = [
                {"role": "user", "content": task},
                {"role": "user", "content": f"```python\n{cell_src}\n```"},
            ]
            )
        except Exception as e:
            if 'maximum context length' in str(e):
                print('...Error.. too long...' + str(e))
                return 'length', None
            else:
                print('...Error.. trying again...' + str(e))
        else:
            break
    return completion.choices[0].finish_reason, completion.choices[0].message["content"]

gpt_results = []
for i, cell_src in enumerate(random_cells):
    print(f'Processing file {i}')
    finish_reason, result = format_code(cell_src)
    print(f'File {i} - {finish_reason}')
    gpt_results.append({'reason': finish_reason, 'result': result})

# save the results to a file
with open(GPT_SAVED_FILE_NAME, 'w') as f:
    f.write(str(gpt_results))

Processing file 0
File 0 - stop
Processing file 1
File 1 - stop
Processing file 2
File 2 - stop
Processing file 3
File 3 - stop
Processing file 4
File 4 - stop
Processing file 5
File 5 - stop
Processing file 6
File 6 - stop
Processing file 7
File 7 - stop
Processing file 8
File 8 - stop
Processing file 9
File 9 - stop
Processing file 10
File 10 - stop
Processing file 11
File 11 - stop
Processing file 12
File 12 - stop
Processing file 13
File 13 - stop
Processing file 14
File 14 - stop
Processing file 15
File 15 - stop
Processing file 16
File 16 - stop
Processing file 17
File 17 - stop
Processing file 18
File 18 - stop
Processing file 19
File 19 - stop
Processing file 20
File 20 - stop
Processing file 21
File 21 - stop
Processing file 22
File 22 - stop
Processing file 23
File 23 - stop
Processing file 24
File 24 - stop
Processing file 25
File 25 - stop
Processing file 26
File 26 - stop
Processing file 27
File 27 - stop
Processing file 28
File 28 - stop
Processing file 29
File 29 - stop


In [7]:
# read in gpt result from file
with open(GPT_SAVED_FILE_NAME, 'r') as f:
    gpt_results = eval(f.read())

In [8]:
sys.path.append('../')
from common import pycodestyle, group_by_error, print_num_reductions, print_percentage_difference, unpack_gpt, IGNORE_TYPES

In [9]:
gpt_formatted_code, gpt_changes = unpack_gpt(gpt_results)

In [10]:
# print all to new folder
import os
if not os.path.exists(GPT_SAVED_FOLDER_NAME):
    os.makedirs(GPT_SAVED_FOLDER_NAME)
for i, code in enumerate(gpt_formatted_code):
    with open(f'{GPT_SAVED_FOLDER_NAME}/{i}.py', 'w') as f:
        if gpt_formatted_code[i] is None or gpt_changes[i] is None:
            f.write(random_cells[i])
        else:
            f.write(code)

In [11]:
# store error counts in a hash
error_counts_before = pycodestyle(FOLDER_NAME, NUM_FILES, IGNORE_TYPES)

# print the error counts
total_errors_before = sum(error_counts_before.values())
print(f'Total before: {total_errors_before}')

Total before: 3062


In [12]:
# store error counts in a hash
error_counts_after = pycodestyle(GPT_SAVED_FOLDER_NAME, NUM_FILES, IGNORE_TYPES)

# print the error counts
total_errors_after = sum(error_counts_after.values())
print(f'Total after: {total_errors_after}')

Total after: 692


In [13]:
error_counts_before

{'E501': 447,
 'E226': 181,
 'E231': 592,
 'E266': 80,
 'E303': 8,
 'W293': 213,
 'E251': 359,
 'E265': 130,
 'W291': 142,
 'E202': 29,
 'E203': 47,
 'E241': 16,
 'E225': 154,
 'E117': 2,
 'E128': 33,
 'E302': 48,
 'E401': 2,
 'E703': 52,
 'E221': 76,
 'E305': 27,
 'E228': 5,
 'E402': 32,
 'E111': 141,
 'E114': 22,
 'E261': 46,
 'E722': 2,
 'E201': 29,
 'E275': 9,
 'E126': 4,
 'E127': 3,
 'E211': 13,
 'E271': 2,
 'E262': 27,
 'E113': 2,
 'E116': 8,
 'W504': 6,
 'E123': 2,
 'E101': 1,
 'W191': 2,
 'E701': 3,
 'E121': 33,
 'E131': 3,
 'E702': 9,
 'E741': 2,
 'E115': 3,
 'E124': 1,
 'E222': 8,
 'E301': 2,
 'E712': 1,
 'E122': 3}

In [14]:
error_counts_after

{'E501': 459,
 'E225': 4,
 'E128': 4,
 'E302': 34,
 'E305': 15,
 'W293': 56,
 'E266': 18,
 'E228': 1,
 'E722': 3,
 'E226': 18,
 'W291': 30,
 'E275': 1,
 'W504': 2,
 'E203': 1,
 'E231': 1,
 'E261': 3,
 'E262': 2,
 'E265': 6,
 'E402': 18,
 'E741': 2,
 'E127': 5,
 'E124': 1,
 'E241': 3,
 'E301': 1,
 'E712': 1,
 'E251': 2,
 'E303': 1}

In [15]:
# List percentage difference between before and after for total
print(f'Total percentage difference: {(total_errors_after - total_errors_before) / total_errors_before * 100}%')

Total percentage difference: -77.40039190071847%


In [16]:
error_counts_grouped_before = group_by_error(error_counts_before)
error_counts_grouped_before

{'E1': 261,
 'E2': 1803,
 'E3': 85,
 'E4': 34,
 'E5': 447,
 'E7': 69,
 'E9': 0,
 'W1': 2,
 'W2': 355,
 'W3': 0,
 'W5': 6,
 'W6': 0}

In [17]:
error_counts_grouped_after = group_by_error(error_counts_after)
error_counts_grouped_after

{'E1': 10,
 'E2': 60,
 'E3': 51,
 'E4': 18,
 'E5': 459,
 'E7': 6,
 'E9': 0,
 'W1': 0,
 'W2': 86,
 'W3': 0,
 'W5': 2,
 'W6': 0}

In [18]:
print_num_reductions(error_counts_grouped_before, error_counts_grouped_after)

('E1', 'Indentation'): 261 -> 10
('E2', 'Whitespace'): 1803 -> 60
('E3', 'Blank line'): 85 -> 51
('E4', 'Import'): 34 -> 18
('E5', 'Line length'): 447 -> 459
('E7', 'Statement'): 69 -> 6
('E9', 'Runtime'): 0 -> 0
('W1', 'Indentation warning'): 2 -> 0
('W2', 'Whitespace warning'): 355 -> 86
('W3', 'Blank line warning'): 0 -> 0
('W5', 'Line break warning'): 6 -> 2
('W6', 'Deprecation warning'): 0 -> 0


In [19]:
print_percentage_difference(error_counts_grouped_before, error_counts_grouped_after)

('E1', 'Indentation'): -96.17%
('E2', 'Whitespace'): -96.67%
('E3', 'Blank line'): -40.0%
('E4', 'Import'): -47.06%
('E5', 'Line length'): 2.68%
('E7', 'Statement'): -91.3%
('E9', 'Runtime'): Undefined
('W1', 'Indentation warning'): -100.0%
('W2', 'Whitespace warning'): -75.77%
('W3', 'Blank line warning'): Undefined
('W5', 'Line break warning'): -66.67%
('W6', 'Deprecation warning'): Undefined


In [ ]:
# delete the folder reformatted_gpt
!rm -rf reformatted_gpt